<a href="https://colab.research.google.com/github/thanitaja/eiei/blob/master/ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten

import os
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "./sample_data"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

000bec180eb18c7604dcecc8fe0dba07.jpg
001cdf01b096e06d78e9e5112d419397.jpg
003df8b8a8b05244b1d920bb6cf451f9.jpg
006cc3ddb9dc1bd827479569fcdc52dc.jpg
008b1271ed1addaccf93783b39deab45.jpg
008ba178d6dfc1a583617470d19c1673.jpg
00a338a92e4e7bf543340dc849230e75.jpg
00a366d4b4a9bbb6c8a63126697b7656.jpg
00a862390341c5be090dd72bd2bc19ef.jpg
00b7d114bc5166a629a3cc03d9329120.jpg
00ba244566e36e0af3d979320fd3017f.jpg
00bee065dcec471f26394855c5c2f3de.jpg
00ca18751837cd6a22813f8e221f7819.jpg
00cc68a50b2d016a6b29af628ea4e04b.jpg
00ddcec076073cc96f82c27bf4548fbc.jpg
00e9ed3fab1d2032603d1a90e557976f.jpg
00fa641312604199831755f96109fde7.jpg
00fda6ecca54efbac26e907be4b0b78b.jpg
010d65bd29d246aea53d9849da142ccf.jpg
010e87fdf252645a827e37470e65e842.jpg
011bd7fd7c036dbb243d0e37329c77b7.jpg
011e06760794850c40f23d6426c40774.jpg
012ff2d21dad14452ea16b4cda7eef4c.jpg
013aed490e0e15a8989e12ae0d0ccfaa.jpg
013c43f40f0fb13fa25ac0c2a70fd48f.jpg
013f8fdf6d638c7bb042f5f17e8a9fdc.jpg
014c2b0cd8e3b517e649cecf8543b8fe.jpg
0

In [0]:
df_train = pd.read_csv('./sample_data/labels.csv')
df_test = pd.read_csv('./sample_data/sample_submission.csv')

In [0]:
df_train.head(10)

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever
5,002211c81b498ef88e1b40b9abf84e1d,bedlington_terrier
6,00290d3e1fdd27226ba27a8ce248ce85,bedlington_terrier
7,002a283a315af96eaea0e28e7163b21b,borzoi
8,003df8b8a8b05244b1d920bb6cf451f9,basenji
9,0042188c895a2f14ef64a918ed9c7b64,scottish_deerhound


In [0]:
targets_series = pd.Series(df_train['breed'])
one_hot = pd.get_dummies(targets_series, sparse = True)

In [0]:
one_hot_labels = np.asarray(one_hot)

In [0]:
im_size = 90

In [0]:
x_train = []
y_train = []
x_test = []

In [0]:
i = 0 
for f, breed in tqdm(df_train.values):
    img = cv2.imread('../input/train/{}.jpg'.format(f))
    label = one_hot_labels[i]
    x_train.append(cv2.resize(img, (im_size, im_size)))
    y_train.append(label)
    i += 1

  0%|          | 0/10222 [00:00<?, ?it/s]


error: ignored

In [0]:
for f in tqdm(df_test['id'].values):
    img = cv2.imread('../input/test/{}.jpg'.format(f))
    x_test.append(cv2.resize(img, (im_size, im_size)))

In [0]:
y_train_raw = np.array(y_train, np.uint8)
x_train_raw = np.array(x_train, np.float32) / 255.
x_test  = np.array(x_test, np.float32) / 255.

In [0]:
print(x_train_raw.shape)
print(y_train_raw.shape)
print(x_test.shape)

In [0]:
num_class = y_train_raw.shape[1]

In [0]:
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=0.3, random_state=1)

In [0]:
# Create the base pre-trained model
# Can't download weights in the kernel
base_model = VGG19(#weights='imagenet',
    weights = None, include_top=False, input_shape=(im_size, im_size, 3))

# Add a new top layer
x = base_model.output
x = Flatten()(x)
predictions = Dense(num_class, activation='softmax')(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# First: train only the top layers (which were randomly initialized)
for layer in base_model.layers:
    layer.trainable = False

model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
model.summary()

In [0]:
model.fit(X_train, Y_train, epochs=1, validation_data=(X_valid, Y_valid), verbose=1)

In [0]:
preds = model.predict(x_test, verbose=1)